In [ ]:
import geopandas as gpd
import numpy as np
import plotly.express as px
from shapely.geometry import Point

from srai.regionizers import VoronoiRegionizer
from srai.constants import WGS84_CRS
from srai.plotting.folium_wrapper import plot_regions
from srai.utils import geocode_to_region_gdf

## Regionize whole Earth

Basic usage of `VoronoiRegionizer` to cover whole Earth using 6 poles.

In [ ]:
# 6 poles of the Earth
seeds_gdf = gpd.GeoDataFrame(
    {
        "geometry": [
            Point(0, 0),
            Point(90, 0),
            Point(180, 0),
            Point(-90, 0),
            Point(0, 90),
            Point(0, -90),
        ]
    },
    index=[1, 2, 3, 4, 5, 6],
    crs=WGS84_CRS,
)

In [ ]:
seeds_gdf.plot()

In [ ]:
vr = VoronoiRegionizer(seeds=seeds_gdf)

In [ ]:
result_gdf = vr.transform()

In [ ]:
result_gdf

#### Globe view

In [ ]:
fig = px.choropleth(
    result_gdf,
    geojson=result_gdf.geometry,
    locations=result_gdf.index,
    color=result_gdf.index,
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig2 = px.scatter_geo(seeds_gdf, lat=seeds_gdf.geometry.y, lon=seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="white", marker_size=10, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="orthographic",
    projection_rotation_lon=20,
    projection_rotation_lat=30,
    showlakes=False,
)
fig.update_layout(height=800, width=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

#### 2D OSM View

In [ ]:
folium_map = plot_regions(result_gdf)
seeds_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=3),
)

## Regionize single country

Drawing a list of arbitrary points inside of the country boundary and using them for regionization of the same geometry.

In [ ]:
uk_gdf = geocode_to_region_gdf(query=["R62149"], by_osmid=True)

uk_shape = uk_gdf.iloc[0].geometry  # get the Polygon

In [ ]:
uk_gdf

In [ ]:
def generate_random_points(shape, n_points=100):
    minx, miny, maxx, maxy = shape.bounds
    pts = []

    while len(pts) < 4:
        randx = np.random.uniform(minx, maxx, n_points)
        randy = np.random.uniform(miny, maxy, n_points)
        coords = np.vstack((randx, randy)).T

        # use only the points inside the geographic area
        pts = [p for p in list(map(Point, coords)) if p.within(shape)]

    del coords  # not used any more

    return pts

In [ ]:
pts = generate_random_points(uk_shape)

uk_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs=WGS84_CRS,
)

#### Random points on a map

In [ ]:
folium_map = plot_regions(uk_gdf, tiles_style="CartoDB positron")
uk_seeds_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=3),
)

In [ ]:
vr_uk = VoronoiRegionizer(seeds=uk_seeds_gdf)

In [ ]:
uk_result_gdf = vr_uk.transform(gdf=uk_gdf)

In [ ]:
uk_result_gdf.head()

#### Generated regions on a map

In [ ]:
folium_map = plot_regions(uk_result_gdf, tiles_style="CartoDB positron")
uk_seeds_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=3),
)

## Higher amount of points

Example of railway stations in Germany (5000+ seeds) with multiprocessing.

In [ ]:
stations_csv = gpd.pd.read_csv(
    "https://raw.githubusercontent.com/trainline-eu/stations/master/stations.csv",
    sep=";",
    index_col="id",
    usecols=["id", "latitude", "longitude", "country"],
)
stations_csv

In [ ]:
stations = []
positions = set()
for idx, r in stations_csv.iterrows():
    if r.country != "DE" or gpd.pd.isna(r.latitude) or gpd.pd.isna(r.longitude):
        continue
    pos = round(r.longitude, 5), round(r.latitude, 5)
    if not pos in positions:
        stations.append({"id": idx, "geometry": Point(*pos)})
        positions.add(pos)

stations_gdf = gpd.GeoDataFrame(data=stations, crs=WGS84_CRS).set_index("id")

del stations_csv
del stations
del positions

stations_gdf.head()

In [ ]:
vr_rail = VoronoiRegionizer(seeds=stations_gdf)

In [ ]:
rail_result_gdf = vr_rail.transform()

#### Germany view

In [ ]:
folium_map = plot_regions(rail_result_gdf, tiles_style="CartoDB positron")
stations_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=1),
)
folium_map.fit_bounds([(54.98310, 5.98865), (47.30248, 15.01699)])
folium_map

#### Berlin view

In [ ]:
# Berlin
folium_map.fit_bounds([(52.51637 + 0.1, 13.40665 + 0.1), (52.51637 - 0.1, 13.40665 - 0.1)])
folium_map

## Difference between spherical voronoi and 2d voronoi

Showing the difference between working on the sphere and projected 2D plane.

Uses `shapely.voronoi_polygons` function as an example.

`shapely.voronoi_diagram` function allows for a quick division of the Earth using list of seeds on a projected 2d plane.
This results in straight lines with angles distorted and polygons differences
might be substantial during comparisons or any area calculations.

In [ ]:
from shapely.ops import voronoi_diagram
from plotly.subplots import make_subplots

In [ ]:
pl_gdf = geocode_to_region_gdf(query=["R49715"], by_osmid=True)

pl_gdf_shape = pl_gdf.iloc[0].geometry  # get the Polygon

In [ ]:
pts = generate_random_points(pl_gdf_shape)

pl_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs=WGS84_CRS,
)

In [ ]:
region_polygons = list(
    voronoi_diagram(pl_seeds_gdf.geometry.unary_union, envelope=pl_gdf_shape).normalize().geoms
)

In [ ]:
pl_regions_2d_gdf = gpd.GeoDataFrame(
    {"geometry": [polygon for polygon in region_polygons]},
    index=list(range(len(region_polygons))),
    crs=WGS84_CRS,
).clip(pl_gdf_shape)

In [ ]:
pl_regions_2d_gdf

In [ ]:
vr_pl = VoronoiRegionizer(seeds=pl_seeds_gdf)

In [ ]:
pl_result_gdf = vr_pl.transform(gdf=pl_gdf)

In [ ]:
pl_result_gdf

In [ ]:
choropleth_1 = px.choropleth(
    pl_result_gdf,
    geojson=pl_result_gdf.geometry,
    locations=pl_result_gdf.index,
    color=pl_result_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

choropleth_2 = px.choropleth(
    pl_regions_2d_gdf,
    geojson=pl_regions_2d_gdf.geometry,
    locations=pl_regions_2d_gdf.index,
    color=pl_regions_2d_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

points_plot = px.scatter_geo(pl_seeds_gdf, lat=pl_seeds_gdf.geometry.y, lon=pl_seeds_gdf.geometry.x)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
    ],
    horizontal_spacing=0.01,
    vertical_spacing=0.0,
)

fig.add_trace(choropleth_1["data"][0], row=1, col=1)
fig.add_trace(choropleth_1["data"][0], row=2, col=1)
fig.add_trace(choropleth_2["data"][0], row=1, col=2)
fig.add_trace(choropleth_2["data"][0], row=2, col=2)
for r in [1, 2]:
    for c in [1, 2]:
        fig.add_trace(points_plot.data[0], row=r, col=c)

minx, miny, maxx, maxy = pl_gdf_shape.bounds

fig.update_traces(marker_color="black", marker_size=6, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
    resolution=50,
    row=1,
    showlakes=False,
)

fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny + 1, maxy - 1],
    lonaxis_range=[minx + 2, maxx - 2],
    resolution=50,
    row=2,
    showlakes=False,
)

fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"), row=1)
fig.update_traces(marker={"opacity": 0.25}, selector=dict(type="choropleth"), row=2)

fig.update_layout(
    height=800,
    width=800,
    title_text="Side By Side Subplots (Left: Spherical voronoi, Right: 2D voronoi)",
    margin={"r": 5, "t": 50, "l": 5, "b": 0},
)
fig.show(renderer="png")  # replace with fig.show() to allow interactivity